In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from scipy import stats
from scipy.stats import norm, skew

In [4]:
import seaborn as sb

In [5]:
# Detect different kinds of Missing values
miss_val=['NaN']

In [13]:
aisles=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)
#aisles.isnull().values.any()

In [17]:
departments=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)
#departments.isnull().values.any()

In [19]:
products=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
#products.isnull().values.any()
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


In [20]:
order_products_prior=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)
#order_products_prior[order_products_prior['order_id']==2774568]
#order_products_prior.isnull().sum()
order_products_prior.shape

(32434489, 4)

In [21]:
order_products_train=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)
#order_products_train.isnull().sum()
order_products_train.shape

(1384617, 4)

In [22]:
orders=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)
#orders.isnull().sum()
#orders.head(10)
orders.shape

(3421083, 7)

In [177]:
#orders[orders['order_id']==2539329] # get info of order having orderid given

In [23]:
orders.fillna(-1, inplace=True) # Fill missing values with -1

In [24]:
# Merging dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [25]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
order_products_usr.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [26]:
order_products_usr['user_id'].max()

206209

In [27]:
# GET A USER'S MOST FREQUENT PRODUCT ORDERED
most_freq_product_user=dict()
for i in range(1,2): #for i in range (1, order_products_usr['user_id'].max()+1):
    most_freq_product_user[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    print("for user %d The size is %d " %(i, most_freq_product_user[i].size))
    print(most_freq_product_user[i])
    
    for j in(most_freq_product_user[i]):
        print(j)

#products[products['product_id']==196] get the product based on the products id

for user 1 The size is 19 
196      11
12427    10
10258    10
25133     9
46149     4
13032     4
26088     3
26405     3
49235     3
38928     2
13176     2
39657     2
41787     1
27845     1
10326     1
17122     1
35951     1
30450     1
14084     1
Name: product_id, dtype: int64
11
10
10
9
4
4
3
3
3
2
2
2
1
1
1
1
1
1
1


In [28]:
# AVERAGE NUMBER OF PRODUCTS BOUGHT ON EVERY ORDER
avg_prod_unique_order=dict()
#for a in range(1, 2):
#    avg_prod_unique_order[a]=order_products_usr[order_products_usr[[order_products_usr['user_id']==a]['order_number']==a]].value_counts()